In [52]:
import os
import sys
import numpy as np
import nibabel as nib
from tqdm import tqdm
# from nilearn.image import resample_img
# Kaiwen's utility functions for plotting in correct orientation
func_dir = '/home/local/VANDERBILT/litz/github/MASILab/thoraxtools/func'
if func_dir not in sys.path:
    sys.path.append(func_dir)
print(sys.path)
# import plot_clip_combined as plo
import vis.paral_clip_overlay_mask as overlay
import vis.vis as vis
import importlib
# importlib.reload(vis.paral_clip_overlay_mask)


['/home-nfs2/local/VANDERBILT/litz/github/MASILab/lobe_seg/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home-nfs2/local/VANDERBILT/litz/github/MASILab/lobe_seg/env/lib/python3.8/site-packages', '/home/local/VANDERBILT/litz/github/MASILab/thoraxtools/func', '/home/local/VANDERBILT/litz/github/MASILab/lobe_seg', '/home/local/VANDERBILT/litz/github/MASILab/lobe_seg', '/home/local/VANDERBILT/litz/github/MASILab/lobe_seg', '/home/local/VANDERBILT/litz/github/MASILab/lobe_seg', '/home/local/VANDERBILT/litz/github/MASILab/lobe_seg', '/home/local/VANDERBILT/litz/github/MASILab/lobe_seg']


In [3]:
def save_clip(raw_path, allmask_path, clip_dir, scanid, tag):
    overlay.multiple_clip_overlay_with_mask(raw_path, allmask_path, os.path.join(clip_dir, f"{tag}_{scanid}_axial.png"), 'axial')
    overlay.multiple_clip_overlay_with_mask(raw_path, allmask_path, os.path.join(clip_dir, f"{tag}_{scanid}_coronal.png"), 'coronal')
    overlay.multiple_clip_overlay_with_mask(raw_path, allmask_path, os.path.join(clip_dir, f"{tag}_{scanid}_sagittal.png"), 'sagittal')

In [26]:
# LSM figure
raw_dir = "/home/local/VANDERBILT/litz/data/imagevu/nifti/train/"
# raw_dir = "/nfs/masi/SPORE/nifti/combine/00001003/20180222/"
resizedraw_dir = "/home/local/VANDERBILT/litz/data/imagevu/lobe/resize_raw"
out_dir = "/home/local/VANDERBILT/litz/github/MASILab/emphysema/figures/"
mask_dir = "/home/local/VANDERBILT/litz/data/imagevu/lobe/lvlsetsegCC/"
scanid="00001003time20180222"
tag = "lvlsetseg"

In [53]:
# johof figure
raw_dir = "/home/local/VANDERBILT/litz/data/imagevu/nifti/train/"
# raw_dir = "/nfs/masi/SPORE/nifti/combine/00001003/20180222/"
resizedraw_dir = "/home/local/VANDERBILT/litz/data/imagevu/lobe/resize_raw"
out_dir = "/home/local/VANDERBILT/litz/github/MASILab/emphysema/figures/"
mask_dir = "/nfs/masi/SPORE/lobe/JoHof/fusedswap/"
scanid="00001003time20180222"
tag="lobe"

In [ ]:

mask_path = os.path.join(mask_dir, f"{tag}_{scanid}.nii.gz")
raw_path = os.path.join(raw_dir, f"{scanid}.nii.gz")
# resize raw to mask shape if not already
rawpath=""
resizedraw_name = f"resize_{scanid}.nii.gz"
resizeraw_path = os.path.join(resizedraw_dir, resizedraw_name)
resizedraws = os.listdir(resizedraw_dir)
# if resizedraw_name not in resizedraws:
#     rawnii = nib.load(raw_path)
#     masknii = nib.load(mask_path)
#     resizeraw_nii = resample_img(rawnii, target_affine=masknii.affine, target_shape=masknii.shape)
#     nib.save(resizeraw_nii, resizeraw_path)

save_clip(resizeraw_path, mask_path, out_dir, scanid, tag)

reading /home/local/VANDERBILT/litz/data/imagevu/lobe/resize_raw/resize_00001003time20180222.nii.gz
reading /home/local/VANDERBILT/litz/github/MASILab/emphysema/figures/unet_00001003time20180222.nii.gz


In [73]:
# unet figure
import sys
import os
sys.path.append("/home/local/VANDERBILT/litz/github/MASILab/lobe_seg")
from dataloader import val_dataloader
from models import unet512, unet1024
import yaml
import torch
import nibabel as nib
from monai.inferers import sliding_window_inference
from monai.handlers.utils import from_engine
from monai.utils import set_determinism
from monai.transforms import (
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,
    EnsureTyped,
    AsDiscrete,
    Invertd,
    AsDiscreted
)
from monai.data import DataLoader, Dataset
from monai.data import decollate_batch

In [78]:
CONFIG_DIR = "/home/local/VANDERBILT/litz/github/MASILab/lobe_seg/configs"
DATA_DIR = "/home/local/VANDERBILT/litz/data/imagevu/nifti/train_preproc/"
out_dir = "/home/local/VANDERBILT/litz/github/MASILab/emphysema/figures/"
config_id = "0322unet512"
scanid="00001003time20180222"
tag="unet"

def load_config(config_name, config_dir):
    with open(os.path.join(config_dir, config_name)) as file:
        config = yaml.load(file, Loader=yaml.FullLoader)
    return config
config = load_config(f"Config_{config_id}.YAML", CONFIG_DIR)
LABEL_DIR = config["label_dir"]
val_images = [os.path.join(DATA_DIR, f"{scanid}.nii.gz")]
val_file_names = [f"lvlsetseg_{os.path.basename(name)}" for name in val_images]
val_labels = [os.path.join(LABEL_DIR, name) for name in val_file_names]
val_files = [
        {"image": image_name, "label": label_name}
        for image_name, label_name in zip(val_images, val_labels)
    ]
# val_loader = val_dataloader(config, images)

hu_window = config["window"] 
val_transforms = Compose([
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Orientationd(keys=["image"], axcodes="RAS"),
        Spacingd(keys=["image"], pixdim=(1.0,1.0,1.0), mode=("bilinear")),
        ScaleIntensityRanged(keys=["image"], a_min=hu_window[0], a_max=hu_window[1], b_min=0.0, b_max=1.0,
                                 clip=True),
        CropForegroundd(keys=["image"], source_key="image"),
        EnsureTyped(keys=["image", "label"]),
    ])
val_ds = Dataset(data=val_files, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=1, num_workers=4)
post_transforms = Compose([
    EnsureTyped(keys=["pred"]),
    Invertd(
        keys=["pred"],
        transform=val_transforms,
        orig_keys="image",
        meta_keys="pred_meta_dict",
        orig_meta_keys="image_meta_dict",
        meta_key_postfix="meta_dict",
        nearest_interp=False,
        to_tensor=True,
    ),
    AsDiscreted(keys="pred", argmax=True),
])


In [79]:


os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
device = torch.device("cuda:0")
model = unet512(6).to(device)
best_model_path = os.path.join(config["model_dir"], config_id, f"{config_id}_best_model.pth")
model.load_state_dict(torch.load(best_model_path))
model.eval()
with torch.no_grad():
    val_data = next(iter(val_loader))
    roi_size = (160, 160, 160)
    sw_batch_size = 4
    val_data["pred"] = sliding_window_inference(val_data["image"].to(device), roi_size, sw_batch_size, model)
    
    
#     output_nii = nib.Nifti1Image(output, header=nii.header, affine=nii.affine)
#     nib.save(output_nii, os.path.join(out_dir, f"{tag}_{scanid}.nii.gz"))


In [80]:
print(val_data["pred"].shape)
val_data = [post_transforms(i) for i in decollate_batch(val_data)]



torch.Size([1, 6, 360, 276, 321])


In [81]:
# unet figure
raw_dir = "/home/local/VANDERBILT/litz/data/imagevu/nifti/train/"
# raw_dir = "/nfs/masi/SPORE/nifti/combine/00001003/20180222/"
resizedraw_dir = "/home/local/VANDERBILT/litz/data/imagevu/lobe/resize_raw"
out_dir = "/home/local/VANDERBILT/litz/github/MASILab/emphysema/figures/"
mask_dir = "/home/local/VANDERBILT/litz/github/MASILab/emphysema/figures/"
scanid="00001003time20180222"
tag="unet"
print(val_data[0]["pred"].shape)
print(val_data[0]["image"].shape)
val_output, val_image = val_data[0]["pred"], val_data[0]["image"]
# np_img = val_image[0, :,:,:].detach().cpu().numpy()
np_img = nib.load(os.path.join(raw_dir, f"{scanid}.nii.gz")).get_fdata()
output = val_output[0,:,:,:].detach().cpu().numpy()
overlay.multiple_clip_overlay_with_mask_from_npy(np_img, output, os.path.join(out_dir, f"{tag}_{scanid}_coronal.png"), clip_plane='coronal')
overlay.multiple_clip_overlay_with_mask_from_npy(np_img, output, os.path.join(out_dir, f"{tag}_{scanid}_sagittal.png"), clip_plane='sagittal')
overlay.multiple_clip_overlay_with_mask_from_npy(np_img, output, os.path.join(out_dir, f"{tag}_{scanid}_axial.png"), clip_plane='axial')
# nii = nib.load(os.path.join(DATA_DIR, f"{scanid}.nii.gz"))
# output_nii = nib.Nifti1Image(output, header=nii.header, affine=nii.affine)

# nib.save(output_nii, os.path.join(out_dir, f"{tag}_{scanid}.nii.gz"))

torch.Size([1, 512, 512, 401])
torch.Size([1, 360, 276, 321])
Save to /home/local/VANDERBILT/litz/github/MASILab/emphysema/figures/unet_00001003time20180222_coronal.png
Save to /home/local/VANDERBILT/litz/github/MASILab/emphysema/figures/unet_00001003time20180222_sagittal.png
Save to /home/local/VANDERBILT/litz/github/MASILab/emphysema/figures/unet_00001003time20180222_axial.png


In [83]:
# emphysema mask
nii = nib.load(os.path.join(raw_dir, f"{scanid}.nii.gz"))
output_nii = nib.Nifti1Image(output, header=nii.header, affine=nii.affine)
nib.save(output_nii, os.path.join(out_dir, f"{tag}_{scanid}.nii.gz"))


In [84]:
emp_path = "/nfs/masi/SPORE/lobe/empmasks"
emp_nii =nib.load(os.path.join(emp_path, f"emp_{scanid}.nii.gz"))
emp_img = emp_nii.get_fdata()
output_x_emp = np.multiply(output, emp_img)
noemp = output - output_x_emp
emp_scaled = np.multiply(emp_img, 6)
all_masks = noemp + emp_scaled
all_masks_nii = nib.Nifti1Image(all_masks, header=emp_nii.header, affine=emp_nii.affine)
nib.save(all_masks_nii, os.path.join(out_dir, f"emp_merged_{scanid}.nii.gz"))

In [87]:
# merged figure
raw_dir = "/home/local/VANDERBILT/litz/data/imagevu/nifti/train/"
# raw_dir = "/nfs/masi/SPORE/nifti/combine/00001003/20180222/"
resizedraw_dir = "/home/local/VANDERBILT/litz/data/imagevu/lobe/resize_raw"
out_dir = "/home/local/VANDERBILT/litz/github/MASILab/emphysema/figures/"
mask_dir = "/home/local/VANDERBILT/litz/github/MASILab/emphysema/figures/"
scanid="00001003time20180222"
tag="emp_merged"
mask_path = os.path.join(out_dir, f"emp_merged_{scanid}.nii.gz")
raw_path = os.path.join(raw_dir, f"{scanid}.nii.gz")
# resize raw to mask shape if not already
resizedraw_name = f"resize_{scanid}.nii.gz"
resizeraw_path = os.path.join(resizedraw_dir, resizedraw_name)
resizedraws = os.listdir(resizedraw_dir)
# if resizedraw_name not in resizedraws:
#     rawnii = nib.load(raw_path)
#     masknii = nib.load(mask_path)
#     resizeraw_nii = resample_img(rawnii, target_affine=masknii.affine, target_shape=masknii.shape)
#     nib.save(resizeraw_nii, resizeraw_path)

save_clip(resizeraw_path, mask_path, out_dir, scanid, tag)

reading /home/local/VANDERBILT/litz/data/imagevu/lobe/resize_raw/resize_00001003time20180222.nii.gz
reading /home/local/VANDERBILT/litz/github/MASILab/emphysema/figures/emp_merged_00001003time20180222.nii.gz
Save to /home/local/VANDERBILT/litz/github/MASILab/emphysema/figures/emp_merged_00001003time20180222_fused_axial.png
reading /home/local/VANDERBILT/litz/data/imagevu/lobe/resize_raw/resize_00001003time20180222.nii.gz
reading /home/local/VANDERBILT/litz/github/MASILab/emphysema/figures/emp_merged_00001003time20180222.nii.gz
Save to /home/local/VANDERBILT/litz/github/MASILab/emphysema/figures/emp_merged_00001003time20180222_coronal.png
reading /home/local/VANDERBILT/litz/data/imagevu/lobe/resize_raw/resize_00001003time20180222.nii.gz
reading /home/local/VANDERBILT/litz/github/MASILab/emphysema/figures/emp_merged_00001003time20180222.nii.gz
Save to /home/local/VANDERBILT/litz/github/MASILab/emphysema/figures/emp_merged_00001003time20180222_sagittal.png
